In [1]:
# Importación de bibliotecas necesarias
import pandas as pd
import numpy as np
import pickle

In [3]:
# Carga de los datos desde archivos CSV
df_games = pd.read_csv('dataset\df_games_limpio.csv')
df_userdata = pd.read_csv('datasetAPI\df_user_data.csv')
df_items = pd.read_csv('dataset\df_items_limpio.csv')

In [11]:
df_items.head()

,user_id,user_url,items_count,steam_id,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,277,76561197970982479,10,counter-strike,6,0
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,277,76561197970982479,20,team fortress classic,0,0
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,277,76561197970982479,30,day of defeat,7,0
3,76561197970982479,http://steamcommunity.com/profiles/76561197970...,277,76561197970982479,40,deathmatch classic,0,0
4,76561197970982479,http://steamcommunity.com/profiles/76561197970...,277,76561197970982479,50,half-life: opposing force,0,0


In [4]:
# crear un dataframe que relaciona cada juego con el tiempo total que han invertido sus jugadores en el
df_most_played=df_items[['item_id','playtime_forever']].groupby('item_id').sum().reset_index()
df_most_played.to_csv('datasetAPI\most_played_games.csv', index=False)

In [5]:
# Creación de un diccionario que mapea ID de juegos a géneros
id_to_genre = df_games.dropna(subset='id').set_index('id')['genres'].to_dict()

In [4]:
# Guardar el diccionario con pickle
with open('datasetAPI\id_to_genre_dict', 'wb') as archivo:
    pickle.dump(id_to_genre, archivo)

In [6]:
# Función para obtener los géneros de un juego a partir de su ID
def get_genres(id):
    return id_to_genre.get(id, [])

In [7]:
id_to_game = df_games.dropna(subset='id').set_index('id')['app_name'].to_dict()

In [10]:
# Guardar el diccionario con pickle
with open('datasetAPI\id_to_game_dict', 'wb') as archivo:
    pickle.dump(id_to_game, archivo)

In [8]:
# Inicialización de una lista para almacenar todos los géneros
generos = []

In [9]:
# Iteración a través de los valores de género en el diccionario
for value in id_to_genre.values():
    # Utiliza la función eval para convertir la cadena de géneros en una lista de Python
    generos.extend(eval(value))


In [10]:
# Obtención de la lista de géneros únicos eliminando duplicados
generos_unicos=list(set(generos))

In [11]:
generos_unicos

['parkour',
 'space',
 'conspiracy',
 'fishing',
 'world war ii',
 'remake',
 'fantasy',
 'economy',
 'dystopian',
 'local co-op',
 'time attack',
 'trackir',
 '6dof',
 'nudity',
 'gamemaker',
 'voice control',
 'rhythm',
 'military',
 'linear',
 'class-based',
 'philisophical',
 'psychological',
 'relaxing',
 'real time tactics',
 'dark comedy',
 'basketball',
 'word game',
 'heist',
 'western',
 'arcade',
 'dynamic narration',
 'martial arts',
 'quick-time events',
 'psychedelic',
 'top-down shooter',
 'exploration',
 'casual',
 'flight',
 'hand-drawn',
 'noir',
 'choose your own adventure',
 'based on a novel',
 'third-person shooter',
 'dungeon crawler',
 'crime',
 'horses',
 'mars',
 'spectacle fighter',
 'grand strategy',
 "shoot 'em up",
 'competitive',
 '2d',
 'chess',
 'steampunk',
 'jrpg',
 'trading card game',
 'fps',
 'grid-based movement',
 'survival',
 'pixel graphics',
 'foreign',
 'hunting',
 'cycling',
 'space sim',
 'surreal',
 'feature film',
 'lemmings',
 'bullet ti

In [12]:
# Función para convertir géneros en variables dummies en una fila del DataFrame
def generos_to_dummies(row):
    generos = row['genre']
    generos_dict = {genero: 1 if genero in generos else 0 for genero in generos_unicos}
    return pd.Series(generos_dict)

In [13]:
# Creación de un DataFrame de usuarios e ítems con variables dummies para géneros
df_user_items=df_items[['user_id', 'item_id']]

In [14]:
# Agrega una columna 'genre' al DataFrame de usuarios e ítems
df_user_items['genre']=df_user_items['item_id'].apply(get_genres)

C:\Users\ARTUROS\AppData\Local\Temp\ipykernel_5088\490258693.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_user_items['genre']=df_user_items['item_id'].apply(get_genres)


In [ ]:
# Aplicación de la función generos_to_dummies a cada fila del DataFrame
df_generos_dummies =df_user_items.apply(generos_to_dummies, axis=1)

In [229]:
# Concatenación de los DataFrames de usuarios e ítems y variables dummies
df_user_items = pd.concat([df_user_items, df_generos_dummies], axis=1)

In [ ]:
# Eliminación de la columna 'item_id' del DataFrame
df_user_items=df_user_items.drop('item_id', axis=1)

In [234]:
# Agrupación de datos por 'user_id' y cálculo del promedio para obtener preferencias promedio por usuario
df_user_items=df_user_items.groupby('user_id').mean()

In [237]:
# Guardado del DataFrame resultante en un archivo CSV
df_user_items.to_csv('dataset/df_user_dummies.csv', index=False)